In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTENC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [2]:
data=pd.read_csv('C:/Users/HCJ/Desktop/2021_Summer/Finance_data/bank.csv',index_col=0)
data.iloc[:,[43,44,45,64,65,66,67,68]] = data.iloc[:,[43,44,45,64,65,66,67,68]].replace({1:0,2:1})
print(data.shape)

(119535, 93)


In [3]:
train_data, test_data = train_test_split(data,test_size=0.2, random_state=100)
print(train_data.shape)
print(test_data.shape)

(95628, 93)
(23907, 93)


In [4]:
X_train = train_data.iloc[:,1:]
y_train = train_data['TARGET']
X_test = test_data.iloc[:,1:]
y_test = test_data['TARGET']

In [5]:
X_train.head()

,J_F1_LON_RK_3M_CNT_RE,J_F1_OTHER_LON_6M_AVBL_AM_RE,J_F1_OTHER_LON_MAX_3M_AM_RE,J_SHINBO_GIBO_AM_RE,J_TOT_LON_CNT_RE,J_TOT_LON_AGG_AM_RE,J_TOT_LON_MAX_AM_RE,J_DPS_CT_BAL_AM_RE,J_TOT_DPS_1M_AVBL_AM_RE,J_TOT_DPS_6M_AVBL_AM_RE,...,P_BR_CNTC_1M_CNT_RE,P_BR_CNTC_6M_CNT_RE,P_ATM_CNTC_6M_CNT_RE,P_INT_CNTC_6M_CNT_RE,P_TOT_LON_CNT_RE,P_TOT_LON_AGG_AM_RE,P_TOT_TS_6M_CNT_RE,P_TOT_TS_6M_AM_RE,P_OB_TS_6M_CNT_RE,P_OB_TS_6M_AM_RE
114140,1,2,3,1,3,3,3,2,2,3,...,1,1,1,1,1,1,1,1,1,1
17613,1,1,2,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
30297,3,3,3,1,1,1,1,1,1,1,...,1,1,3,2,3,3,3,3,3,3
54831,2,3,3,1,1,1,1,2,2,2,...,1,1,1,2,1,1,1,1,1,1
117127,3,3,4,2,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [6]:
y_train.head()

114140    0
17613     0
30297     0
54831     0
117127    0
Name: TARGET, dtype: int64

In [7]:
smotenc = SMOTENC(categorical_features = [42,43,44,63,64,65,66,67])
X_train_samp, y_train_samp = smotenc.fit_resample(X_train,y_train)

In [11]:
#MLP 샘플링 smotenc test1
mlp2 = MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', solver='sgd', alpha=0.01, batch_size=32,learning_rate_init=0.1, max_iter=500)  
mlp2.fit(X_train_samp, y_train_samp)
Y_pred2 = mlp2.predict(X_test)
#성능 평가
mlp_accuracy=metrics.accuracy_score(y_test,Y_pred2)
mlp_score=metrics.precision_recall_fscore_support(y_test,Y_pred2)

print(mlp_accuracy)
print(pd.DataFrame(mlp_score))

0.7634165725519723
              0           1
0      0.998622    0.022735
1      0.762915    0.839744
2      0.864999    0.044272
3  23751.000000  156.000000


In [12]:
#MLP 샘플링 smotenc test2
mlp2 = MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', solver='sgd', alpha=0.01, batch_size=32,learning_rate_init=0.1, max_iter=500)  
mlp2.fit(X_train_samp, y_train_samp)
Y_pred2 = mlp2.predict(X_test)
#성능 평가
mlp_accuracy=metrics.accuracy_score(y_test,Y_pred2)
mlp_score=metrics.precision_recall_fscore_support(y_test,Y_pred2)

print(mlp_accuracy)
print(pd.DataFrame(mlp_score))

0.88689505165851
              0           1
0      0.998154    0.042056
1      0.887794    0.750000
2      0.939745    0.079646
3  23751.000000  156.000000


In [37]:
#MLP bagging test1
from sklearn.ensemble import BaggingClassifier
bag_clf = BaggingClassifier(MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', solver='sgd', alpha=0.01, batch_size=32,learning_rate_init=0.1, max_iter=500),
                           n_estimators=500,
                           max_samples=100,
                           bootstrap=True,
                           n_jobs=-1)
model = bag_clf.fit(X_train_samp, y_train_samp)
y_pred = model.predict(X_test)
#성능 평가
mlp_accuracy=metrics.accuracy_score(y_test,y_pred)
mlp_score=metrics.precision_recall_fscore_support(y_test,y_pred)

print(mlp_accuracy)
print(pd.DataFrame(mlp_score))

0.793533274773079
              0           1
0      0.998305    0.024662
1      0.793524    0.794872
2      0.884213    0.047840
3  23751.000000  156.000000


In [38]:
#MLP bagging test2
from sklearn.ensemble import BaggingClassifier
bag_clf = BaggingClassifier(MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', solver='sgd', alpha=0.01, batch_size=32,learning_rate_init=0.1, max_iter=500),
                           n_estimators=500,
                           max_samples=100,
                           bootstrap=True,
                           n_jobs=-1)
model = bag_clf.fit(X_train_samp, y_train_samp)
y_pred = model.predict(X_test)
#성능 평가
mlp_accuracy=metrics.accuracy_score(y_test,y_pred)
mlp_score=metrics.precision_recall_fscore_support(y_test,y_pred)

print(mlp_accuracy)
print(pd.DataFrame(mlp_score))

0.762203538712511
              0           1
0      0.998565    0.022456
1      0.761736    0.833333
2      0.864219    0.043734
3  23751.000000  156.000000


In [39]:
#AdaBoostClassifier test1
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier(n_estimators=50,
                         learning_rate=1)
ada_clf.fit(X_train_samp, y_train_samp)
y_pred = bag_clf.predict(X_test)
#성능 평가
mlp_accuracy=metrics.accuracy_score(y_test,y_pred)
mlp_score=metrics.precision_recall_fscore_support(y_test,y_pred)

print(mlp_accuracy)
print(pd.DataFrame(mlp_score))

0.762203538712511
              0           1
0      0.998565    0.022456
1      0.761736    0.833333
2      0.864219    0.043734
3  23751.000000  156.000000


In [40]:
#AdaBoostClassifier test2
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier(n_estimators=50,
                         learning_rate=1)
ada_clf.fit(X_train_samp, y_train_samp)
y_pred = bag_clf.predict(X_test)
#성능 평가
mlp_accuracy=metrics.accuracy_score(y_test,y_pred)
mlp_score=metrics.precision_recall_fscore_support(y_test,y_pred)

print(mlp_accuracy)
print(pd.DataFrame(mlp_score))

0.762203538712511
              0           1
0      0.998565    0.022456
1      0.761736    0.833333
2      0.864219    0.043734
3  23751.000000  156.000000


In [48]:
#VotingClassifier test1
from sklearn.ensemble import VotingClassifier
mlp1 = MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', solver='sgd', alpha=0.01, batch_size=32,learning_rate_init=0.1, max_iter=500)
mlp2 = MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', solver='sgd', alpha=0.01, batch_size=32,learning_rate_init=0.1, max_iter=500)
mlp3 = MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', solver='sgd', alpha=0.01, batch_size=32,learning_rate_init=0.1, max_iter=500)
mlp4 = MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', solver='sgd', alpha=0.01, batch_size=32,learning_rate_init=0.1, max_iter=500)

vo_clf=VotingClassifier(estimators=[('M1',mlp1),('M2',mlp2), ('M3',mlp3), ('M4',mlp4)],voting='soft')
vo_clf.fit(X_train_samp, y_train_samp)
y_pred=vo_clf.predict(X_test)
#성능 평가
mlp_accuracy=metrics.accuracy_score(y_test,y_pred)
mlp_score=metrics.precision_recall_fscore_support(y_test,y_pred)

print(mlp_accuracy)
print(pd.DataFrame(mlp_score))

0.7750867946626511
              0           1
0      0.999131    0.025450
1      0.774283    0.897436
2      0.872453    0.049496
3  23751.000000  156.000000


In [49]:
#VotingClassifier test2
from sklearn.ensemble import VotingClassifier
mlp1 = MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', solver='sgd', alpha=0.01, batch_size=32,learning_rate_init=0.1, max_iter=500)
mlp2 = MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', solver='sgd', alpha=0.01, batch_size=32,learning_rate_init=0.1, max_iter=500)
mlp3 = MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', solver='sgd', alpha=0.01, batch_size=32,learning_rate_init=0.1, max_iter=500)
mlp4 = MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', solver='sgd', alpha=0.01, batch_size=32,learning_rate_init=0.1, max_iter=500)

vo_clf=VotingClassifier(estimators=[('M1',mlp1),('M2',mlp2),('M3',mlp3),('M4',mlp4)],voting='soft')
vo_clf.fit(X_train_samp, y_train_samp)
y_pred=vo_clf.predict(X_test)
#성능 평가
mlp_accuracy=metrics.accuracy_score(y_test,y_pred)
mlp_score=metrics.precision_recall_fscore_support(y_test,y_pred)

print(mlp_accuracy)
print(pd.DataFrame(mlp_score))

0.7473961601204668
              0           1
0      0.999211    0.023026
1      0.746326    0.910256
2      0.854450    0.044915
3  23751.000000  156.000000
